<figure>
  <IMG SRC="input/TUM.png" WIDTH=250 ALIGN="right">
</figure>

# Image Classification with Keras
    
*David B. Blumenthal*, *Suryadipto Sarkar*



## What is Tensorflow?


Tensorflow is an open-source end-to-end platform that facilitates designing and deploying Machine Learning models using Python.



## What is Keras?


Keras is an API built on top of TensorFlow, that supports deep learning.

In [ ]:
# IMPORT REQUIRED LIBRARIES:
# --------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import numpy as np
from numpy import genfromtxt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
from PIL import Image
import imageio
import pandas as pd
import random
import pickle
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from skimage.io import imread_collection
import glob

## **Note:**
* This is the way to mount drive and read images directly from Google Drive. However, since we have 24,000 images, this will take a while. Therefore, I will show you just this first step on Spyder as I can access the files locally.
* Then, we can just upload the numpy arrays here and work on those.

  * Anyway, you only need to do this the very first time that you read in the data.
  * .py script can be accessed over this link: 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Classification Dataset


We will make use of the popular 'Cats vs Dogs' classification dataset.

Dataset download link: https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

## Simple exercise:
*(Uncomment the code for answer)*


* Read all of the .jpeg and .png images from a folder.

* Save all of the images in a single list (i.e., a list of image arrays). Note that this is the most common way of working with data in python. We often save all of the data into numpy lists or dataframes or similar structures.

There are many ways of doing this. I have made use of **opencv** as it is one of the most popular Computer Vision libraries in python. Read more at: https://github.com/opencv/opencv

In [ ]:
# Simple way to read in all of the data:
# --------------------------------------

####################################################################################
# CATS_folder='/Users/surya/Downloads/kagglecatsanddogs_3367a (1)/PetImages/Cat' # CATS_folder='Cat_folder_path' | # If reading from Drive: CATS_folder='/content/drive/MyDrive/PetImages/Cat'
# DOGS_folder='/Users/surya/Downloads/kagglecatsanddogs_3367a (1)/PetImages/Dog' # DOGS_folder='Dog_folder_path' | # If reading from Drive: DOGS_folder='/content/drive/MyDrive/PetImages/Dog'

# imdir = CATS_folder # or, DOGS_folder
# ext = ['png', 'jpg'] # add other image formats for other datasets
# files = []
# [files.extend(glob.glob(imdir + '*.' + e)) for e in ext]
# images = [cv2.imread(file) for file in files]
####################################################################################

## Data cleanup:

In real-world imaging datasets, we often require a lot of preprocessing (especially when it comes to medical data-for example, radiological scans or cellular microscopy images).

* In this tutorial, however, we will not focus too much into image pre-processing as this is not an advanced imaging course.

* However, we will give an example of how to remove corrupted files in this particular dataset:


```
* I would advise you to take a look at the os library in python (https://docs.python.org/3/library/os.html).
* It is very handy when you want to interact with the operating system using in-built python functions.
```

In [ ]:
# We will remove the corrupted images by making use of the following code (already provided in the official Keras page for the 'Dogs vs Cats' dataset handling):
# --------------------------------------------------------------------------------------------------------------------------------------------------------------
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    # folder_path = os.path.join("Parent_folder_path", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

## Resize the images:

We will reduce the size of each image. Why?

* To save space, and for quick processing times.

* To avoid overfitting (i.e., providing too much unnecessary information per image)


```
There are many ways of doing this. We will make use of the open-cv library.
```
**Note: It is a very common mistake to just reduce the images without preserving aspect ratio. Do not make this mistake. Be sure to use the correct ratio to reduce image size.**

In [ ]:
IMG_SIZE=32

directory='/Users/surya/Downloads/kagglecatsanddogs_3367a (1)/PetImages'
classes=['Cat', 'Dog']

training_data = []

def LoadImagesAndLabels():
    for c in classes:
        path=os.path.join(directory,c) # path to "0" or "1" directory
        class_num=classes.index(c)
    
    
        for filename in os.listdir(path):
            fpath = os.path.join(path, filename)
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(100)
            if is_jfif:
                img = cv2.imread(os.path.join(path,filename))
            if img is not None:
                img=cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                img=img.tolist()
                training_data.append([img,class_num])
    return training_data
LoadImagesAndLabels()

## Homework:

1. Print the number of images in each class. And the total number of images. Verify that the total sample size equals the sum of samples in each of the individual classes.

    ***Hint: Make use of len() or np.shape()[0] or df.shape[0]***


2. Plot the first image from each of the two classes.

    ***Hint: Read about the matplotlib library used for generating plots in python.***

## Saving the data:

* We often need to run ML algorithms many times with little tweaks in the model.

* How do we achieve this? Of course, we could read the data every time and save it as a list as shown above.

* However, this is a very time-consuming approach.

* What is a better approach? Saving the data as a list of image arrays once, and save that for later use.



Read about the **`.pickle`** and **`.npy`** file types. (Here, we will use pickle - don't forget to import the **** library.)


In [ ]:
# # SYNTAX:
# # -------
# pickle_out=open("data.pickle","wb")
# pickle.dump(data,pickle_out)
# pickle_out.close()

## But first let's randomize the data before saving:

* First option is using the random.shuffle() function

In [ ]:
random.shuffle(training_data)

* Second option (exercise): Implement a function that does this

In [ ]:
X=[]
y=[]

for features, label in training_data:
    X.append(features)
    y.append(label)

def Shuffle(X, y):
    # * ENTER CODE HERE * #
X, y=Shuffle(X, y)

* Solution (uncomment to run):

In [ ]:
# def Shuffle(X, y):
    # X_shuffled=[]
    # y_shuffled=[]
    # length=len(y)
    # index=list(range(length))
    # random.Random(12).shuffle(index)
    # for i in range(length):
        # X_shuffled.append(X[index[i]])
        # y_shuffled.append(y[index[i]])
    # return X_shuffled, y_shuffled

## Finally, save shuffled data as .pickle file:

In [ ]:
# Save the training data
pickle_out=open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()
# Save the training labels
pickle_out=open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

# **CONVOLUTIONAL NEURAL NETWORKS (CNNs):**

# Why CNNs?

* Convert data to embeddings/ features
* Example: Converting images from pixel space to feature space
* Enhances learning, reduces dimensionality, represents the data better

![](https://drive.google.com/uc?export=view&id=1imb0ZgzQ6sK02SniXUf4VebiLL7A14xa)

# Components/ Layers:

### **Question**: Looking at the kernel matrix provided above, what kind of pattern do you think it is meant to detect?

  * Answer: Vertical edges

## I. Convolutional Layer:
* Helps extract local patterns in the data (here, image).
* Also helps reduce the number of features. But that is a byproduct of the convolution operation, it is not the main objective. The main objective is to extract meaningful local patterns.
+ ### **Note: If the image is an RGB (3-channel image), the convolved image is also 3-channel. If the input image is a gray (1-channel) image, the convolved image is also single-channeled.**
  + This is because the kernel is applied on each channel separately for convolution.

<br><br>

## **An oversimplified example of convolution:**
(Note: Kernel size 4*4, padding 0, stride length=1)
![](https://drive.google.com/uc?export=view&id=1QngPcDB6pwkIz5Yhftr-3Rj8WEgD5h0Z)

## II. Pooling Layer:

* The main function of the pooling layer is to reduce dimensionality.
* Two popular types of pooling: MaxPooling, and AveragePooling.
* AveragePooling also helps in noise reduction.

<br>

## **A simple example of Pooling:**
(Note: Pooling window size 3*3)
![](https://drive.google.com/uc?export=view&id=17FzHhVdWnI0nrQDkOKiw-JlLqt0mJ9_p)

## III. Feedforward Layer:
* Standard Neural Network architecture used for classification

<br>

## **A simple fully-connected, feed forward neural network:**

![](https://drive.google.com/uc?export=view&id=1bdz0dUNBzt7Ovo5m57-U_-qiBsq846sE)



## Exercise:
* Multi-class classification requires a Softmax layer, but two-class classification does not. Explain why.

# Note on the 'Flatten' layer:
* This is really not a layer in the conventional sense, although it is defined in the tensorflow.keras.layers.
* This is a function used to convert the features (or weights) after pooling, to be fed into the aforementioned feedforward neural network for classification.

  * We will see this a little later when we design the model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Homework reading:**

### Basic ideas behind machine learning and AI:
* What do we mean by 'learning' and 'intelligent' systems?
* What are the three main types of machine learning, and what are the differences?

### Artificial neural networks:
* NN-related terms: Neurons, layers, activation functions, fully connected networks, multi-layer perceptron
* Hyperparameter tuning and model improvement basics: Learning rate, no. of neurons oer layer, how to set model size (i.e., no. and type of layers)
* Learning-related: Learning rate, backpropagation, gradient descent

### Optional reading (slightly more advanced):
* What is transfer learning? What are pre-trained models, and how to use them? Why pre-trained models?
* What is overfitting? How to 1. detect 2. tackle overfitting?
  * Regularization, Dropout, resampling, oversampling (read about 'SMOTE') and undersampling, data augmentation techniques.

# **Designing the model:**


<br>

### A schematic representation of our model:

![](https://drive.google.com/uc?export=view&id=1bDTsN4kiXvC1gh2vlPeVtbAhgmiu_dBn)



In [ ]:
X=pickle.load(open("X.pickle","rb"))
y=pickle.load(open("y.pickle","rb"))

# Normalize data
X=np.asarray(X)/255.0
# X=X.tolist()

y = np.array(y)

model=Sequential()

model.add(  Conv2D(64,(3,3),input_shape=X.shape[1:])  )
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(  Conv2D(32,(2,2))    )
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('sigmoid'))

# model.add(Dense(64))
# model.add(Activation('sigmoid'))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# history=model.fit(X, y, batch_size=32, shuffle=True, sample_weight=None, epochs=50,validation_split=0.1, verbose = 1, callbacks=[callback]) # seed=100,         

history=model.fit(X, y, batch_size=32, shuffle=True, sample_weight=None, epochs=50,validation_split=0.1, verbose = 1) # seed=100,         


# model.fit(X,y,batch_size=32,epochs=25,validation_split=0.1)

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Follow-up exercise:
* How many neurons are there in each of the aforementioned layers?
  * Answer: Conv-1: 64, Conv2: 32, Dense-1: 128, Dense-2: 1
* Why have we not used a Softmax layer?
  * Answer: Because in 2-class classification, the probabilities will anyway add up to 1.
* Run the model by inverting the no. of neurons in the first two Conv2D layers. How does the model perform (better/ worse)?
* How does the model perform if we make use of stochastic gradient descent optimizer instead of Adam?

### References:
[1] Image generated with BioRender.com